## 1st strategy for E.coli 5-HTP production

In [1]:
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

In [2]:
model = read_sbml_model('../data/iML1515.xml')

**5-HTP synthesis pathway**

5-HTP production (phhA)\
tetrahydromonapterin (MH4) + oxygen + tryptophan (trp) <-> 4a-hydroxytetrahydromonopterin (hydro-MH4) + 5-hydroxytryptophan (5-HTP)

In [3]:
#add metabolites to the system
four_alpha_hydroxytetrahydromonopterin = Metabolite(id='hthpth_c', compartment='c')
hydroxytryptophan = Metabolite(id='htrp_c', compartment='c')

In [4]:
#design reaction
phhA = Reaction('phhA')
phhA.add_metabolites({model.metabolites.thmnp_c: -1,
                      model.metabolites.o2_c: -1,
                      model.metabolites.trp__L_c: -1,
                      four_alpha_hydroxytetrahydromonopterin: 1,
                      hydroxytryptophan: 1
                     })
print(phhA.build_reaction_string())

o2_c + thmnp_c + trp__L_c --> hthpth_c + htrp_c


In [5]:
#add reaction to the system
model.add_reactions([phhA])
model.reactions.phhA

Reaction identifier,phhA
Name,
Memory address,0x07f2e4570e3a0
Stoichiometry,o2_c + thmnp_c + trp__L_c --> hthpth_c + htrp_c O2 O2 + Tetrahydromonapterin + L-Tryptophan --> +
GPR,
Lower bound,0.0
Upper bound,1000.0


**Restioration of MH4**

pterin-4alpha-carbinolamine dehydratase (PCD)\
4a-hydroxytetrahydromonopterin (hydro-MH4) <-> dihydromonapterin (MH2) + water

In [6]:
#design reaction
pcd = Reaction('pcd')
pcd.add_metabolites({four_alpha_hydroxytetrahydromonopterin: -1,
                      model.metabolites.dhmpt_c: 1,
                      model.metabolites.h2o_c: 1,
                     })
print(pcd.build_reaction_string())

hthpth_c --> dhmpt_c + h2o_c


In [7]:
#add reaction to the system
model.add_reactions([pcd])
model.reactions.pcd

Reaction identifier,pcd
Name,
Memory address,0x07f2e457230a0
Stoichiometry,hthpth_c --> dhmpt_c + h2o_c --> Dihydromonapterin + H2O H2O
GPR,
Lower bound,0.0
Upper bound,1000.0


dihydromonapterin reductase (DHMR)\
dihydromonapterin (MH2) + H+ + NADPH <-> NADP+ + tetrahydromonapterin (MH4)

In [8]:
#design reaction
dhmr = Reaction('dhmr')
dhmr.add_metabolites({model.metabolites.h_c: -1,
                      model.metabolites.dhmpt_c: -1,
                      model.metabolites.nadph_c: -1,
                      model.metabolites.nadp_c: 1,
                      model.metabolites.thmnp_c: 1
                     })
print(dhmr.build_reaction_string())

dhmpt_c + h_c + nadph_c --> nadp_c + thmnp_c


**Remove 5-HTP from the system**

In [9]:
five_hydroxytryptophan_exchange = Reaction('EX_htrp')
five_hydroxytryptophan_exchange.add_metabolites({model.metabolites.htrp_c: -1})
model.add_reaction(five_hydroxytryptophan_exchange)

**Optimize model**

In [10]:
print(model.objective)

Maximize
1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685


In [ ]:
# writing the model
write_sbml_model(model, "../data/iML1515_5htp_1st_strategy.xml")